In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
config = {
    'project': 'babble',
    'domain': 'spouse',
    'debug': True,
#     'db_name': 'babble_cdr_featurized_temp',
#     'lf_source': 'gradturk',
#     'max_explanations': 30,
    'seed': 1,
    'parallelism': 1,
    'splits': [0,1,2],
    'disc_model_class': 'logreg',
    'supervision': 'majority',
    'gold_explanations': False,
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_spouse_debug.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

if config['debug']:
    print("NOTE: --debug=True: modifying parameters...")
    config['max_docs'] = 100
    config['gen_model_search_space'] = 2
    config['disc_model_search_space'] = 2
    config['gen_params_default']['epochs'] = 25
    config['disc_params_default']['n_epochs'] = 5

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

Overwriting domain=None to domain=spouse
Overwriting print_freq=1 to print_freq=5
Overwriting LF_acc_prior_weight_default=1.0 to LF_acc_prior_weight_default=0.5
Overwriting decay=0.95 to decay=0.99
Overwriting init_class_prior=0 to init_class_prior=-1.15
Overwriting reg_param=0.1 to reg_param=0.5
Overwriting seed=0 to seed=1
Overwriting babbler_candidate_split=1 to babbler_candidate_split=[0, 1, 2]
Overwriting disc_model_class=lstm to disc_model_class=logreg
Overwriting supervision=generative to supervision=majority
Overwriting debug=False to debug=True
NOTE: --debug=True: modifying parameters...
Using SpousePipeline object.


In [5]:
from snorkel.models import Document

session.query(Document).count()

100

In [6]:
for split in [0,1,2]:
    print(session.query(pipe.candidate_class).filter(
        pipe.candidate_class.split == split).count())

831
89
56


In [7]:
# %time pipe.parse()

In [8]:
# %time pipe.extract()

In [9]:
# %time pipe.load_gold()

In [10]:
# %time pipe.featurize()

In [11]:
%time pipe.collect()

Linking candidates...
# CANDIDATES: 976
Building list of target candidate ids...
Collected 30 unique target candidate ids from 30 explanations.
Gathering desired candidates...
Could not find 30 target candidates with the following stable_ids (first 5):
e7b41f2f-d98a-45a1-9616-1d4a65e24501::span:81:84~~e7b41f2f-d98a-45a1-9616-1d4a65e24501::span:90:104
75e1d890-c481-40fb-8d12-34fe6bcbd56d::span:4417:4422~~75e1d890-c481-40fb-8d12-34fe6bcbd56d::span:4446:4457
d38fa137-ac2d-4e43-b216-18dacf22426d::span:3814:3823~~d38fa137-ac2d-4e43-b216-18dacf22426d::span:3957:3970
e2657a53-566f-49d8-a6cf-8d676fd9c429::span:833:846~~e2657a53-566f-49d8-a6cf-8d676fd9c429::span:852:866
6c242461-04a2-4d76-89b7-ecf20995032f::span:572:593~~6c242461-04a2-4d76-89b7-ecf20995032f::span:751:761
Found 0/30 desired candidates
Linking explanations to candidates...
Linked 0/30 explanations
Calling babbler...
Created grammar with 596 rules
Flushing all parses from previous explanation set.
[================================

In [12]:
# pipe.babbler.get_lf_stats()

In [13]:
# pipe.babbler.filtered_analysis()

In [14]:
# %time pipe.label()

In [15]:
# %time pipe.supervise()

In [16]:
# %time pipe.classify()